# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [51]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [52]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print classes

['Frog', 'HeadBack', 'Left', 'Knee', 'Crouch', 'Back', 'Belly', 'Right', 'Sit', 'Stand', 'StandInit']


In [84]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename))
    target = [i] * len(data)
    return data, target

In [61]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE

print 'total number of data', len(all_data)

total number of data 200


In [86]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [56]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [57]:
# YOUR CODE HERE

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0, degree=3,
  gamma=0.001, kernel='rbf', max_iter=-1, probability=False,
  random_state=None, shrinking=True, tol=0.001, verbose=False)

### predicting

In [58]:
clf.predict(all_data[-1]), all_target[-1]

(array([10]), 10)

In [81]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [82]:
expected = []
predicted = []
# YOUR CODE HERE

evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00         5
          1       1.00      1.00      1.00         7
          2       1.00      1.00      1.00        15
          3       1.00      1.00      1.00         5
          4       1.00      1.00      1.00        20
          5       1.00      1.00      1.00         6
          6       1.00      1.00      1.00         8
          7       1.00      1.00      1.00         7
          8       1.00      1.00      1.00        18
          9       1.00      1.00      1.00         8
         10       1.00      1.00      1.00        41

avg / total       1.00      1.00      1.00       140


Confusion matrix:
[[ 5  0  0  0  0  0  0  0  0  0  0]
 [ 0  7  0  0  0  0  0  0  0  0  0]
 [ 0  0 15  0  0  0  0  0  0  0  0]
 [ 0  0  0  5  0  0  0  0  0  0  0]
 [ 0  0  0  0 20  0  0  0  0  0  0]
 [ 0  0  0  0  0  6  0  0  0  0  0]
 [ 0  0  0  0  0  0  8  0  0  0  0]
 [ 0  0  0  0 

## 4. Evaluate on the test data

In [83]:
expected = []
predicted = []
# YOUR CODE HERE

evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00         5
          1       1.00      1.00      1.00         3
          2       1.00      1.00      1.00         5
          3       1.00      1.00      1.00         5
          4       0.91      1.00      0.95        10
          5       1.00      1.00      1.00         4
          6       1.00      1.00      1.00         2
          7       1.00      0.75      0.86         4
          8       1.00      1.00      1.00         8
          9       1.00      1.00      1.00         3
         10       1.00      1.00      1.00        11

avg / total       0.98      0.98      0.98        60


Confusion matrix:
[[ 5  0  0  0  0  0  0  0  0  0  0]
 [ 0  3  0  0  0  0  0  0  0  0  0]
 [ 0  0  5  0  0  0  0  0  0  0  0]
 [ 0  0  0  5  0  0  0  0  0  0  0]
 [ 0  0  0  0 10  0  0  0  0  0  0]
 [ 0  0  0  0  0  4  0  0  0  0  0]
 [ 0  0  0  0  0  0  2  0  0  0  0]
 [ 0  0  0  0 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [68]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

Then, in the application we can load the trained classifier again.

In [71]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1]), all_target[-1]

(array([10]), 10)